## TODO

- [ ] afficher un fond de carte

In [ ]:
import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
import geoviews as gv
import geoviews.feature as gf

import datashader as ds
import datashader.transfer_functions as tf
from datashader.transfer_functions import shade, stack
from holoviews.operation.datashader import datashade

hv.extension('bokeh', width=95)

In [ ]:
opts.defaults(
    opts.Points(apply_ranges=False, ),
    opts.RGB(width=1200, height=682, xaxis=None, yaxis=None, show_grid=False),
    opts.Shape(fill_alpha=0, line_width=1.5, apply_ranges=False, tools=['tap']),
    opts.WMTS(alpha=0.5)
)

In [ ]:
df = pd.read_csv("out/bornes-IRVE.csv", sep=";")

In [ ]:
df = df[ pd.to_numeric(df['Xlongitude'], errors='coerce').notnull() ]
df = df[ pd.to_numeric(df['Ylatitude'], errors='coerce').notnull() ]
df = df[ pd.to_numeric(df['nbre_pdc'], errors='coerce').notnull() ]

In [ ]:
df['Xlongitude'] = pd.to_numeric(df['Xlongitude'])
df['Ylatitude'] = pd.to_numeric(df['Ylatitude'])
df['nbre_pdc'] = pd.to_numeric(df['nbre_pdc'])

In [ ]:
list(df)

In [ ]:
df.info()

In [ ]:
df[["Xlongitude","Ylatitude","nbre_pdc"]].head()

In [ ]:
bounds = [[-6.152344,42.504503],[8.701172,51.713416]]

cvs = ds.Canvas(plot_width=300, plot_height=300, x_range=(-6.152344,8.701172), y_range=(42.504503,51.713416))
agg = cvs.points(df, 'Xlongitude', 'Ylatitude', ds.count('nbre_pdc'))
img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='log')

img

In [ ]:
tf.Images(tf.shade(ds.Canvas().raster(agg, interpolate='linear')))

In [ ]:
url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.png"
geomap = gv.WMTS(url)
points = hv.Points(gv.Dataset(df, kdims=['Xlongitude', 'Ylatitude']))
irve = dynspread(datashade(points, element_type=gv.Image, aggregator=ds.sum('nbre_pdc')))
geomap * irve